## 6. Command Line Question

As done in the previous assignment, we encourage using the command as a feature that Data Scientists must master.

Note: To answer the question in this section, you must strictly use command line tools. We will reject any other method of response. The final script must be placed in CommandLine.sh.

First, take the course_i.tsv files you created in point 1 and merge them using Linux commands (Hint: make sure that the first row containing the column names appears only once).

Now that you have your merged file named merged_courses.tsv, use Linux commands to answer the following questions:
- Which country offers the most Master's Degrees? Which city?
- How many colleges offer Part-Time education?
- Print the percentage of courses in Engineering (the word "Engineer" is contained in the course's name).

__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.

In [30]:
%%bash
cat TSV/course_1.tsv > merged_courses.tsv
for i in {2..6000}
do
   tail -n +2 TSV/course_${i}.tsv >> merged_courses.tsv
done

In [31]:
%%bash  
echo -n "Country: "
cut -f11 merged_courses.tsv | sort | uniq -c | sort -nr | head -1 | awk '{count=$1; $1=""; sub(/^ /,""); print $0 ", number of occurrences: " count}'
echo -n "City: "
cut -f10 merged_courses.tsv | sort | uniq -c | sort -nr | head -1 | awk '{count=$1; $1=""; sub(/^ /,""); print $0 ", number of occurrences: " count}'


Country: United Kingdom, number of occurrences: 4485


City: London, number of occurrences: 1085


In [32]:
%%bash
part_time=$(awk -F'\t' 'tolower($X) ~ /part[- ]time/ {print $2}' merged_courses.tsv | sort | uniq | wc -l)
echo "Number of universities that give part time: ${part_time}" 

Number of universities that give part time: 153


In [33]:
%%bash
total_courses=$(wc -l < merged_courses.tsv)
engineering_courses=$(awk -F '\t' 'tolower($1) ~ /engineer/' merged_courses.tsv | wc -l)
echo "Percentage of engineering courses: $(echo "scale=2; $engineering_courses * 100 / $total_courses" | bc) %"

Percentage of engineering courses: 10.28 %
